<a href="https://colab.research.google.com/github/deepanrajm/machine_learning/blob/master/Classification/Face_Recognition/Face_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Face Recognition using LBPH**

![](https://drive.google.com/uc?export=view&id=11Y1L6VxaxAgwms5k0Iaw5NSyGq3nTQjj)

![](https://drive.google.com/uc?export=view&id=1axqiIBOiVbbuyCwTJ31xcxfCsWXvZoe-)

In [1]:
!git clone https://github.com/deepanrajm/machine_learning.git

Cloning into 'machine_learning'...
remote: Enumerating objects: 72689, done.
remote: Counting objects: 100% (2659/2659), done.
remote: Compressing objects: 100% (2511/2511), done.
remote: Total 72689 (delta 152), reused 2640 (delta 141), pack-reused 70030
Receiving objects: 100% (72689/72689), 122.27 MiB | 23.19 MiB/s, done.
Resolving deltas: 100% (154/154), done.
Updating files: 100% (5561/5561), done.


In [ ]:
!pip install opencv-python==3.4.3.18

In [11]:
from skimage import feature
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from imutils import paths
import cv2
import numpy as np
import pickle as cPickle
from google.colab.patches import cv2_imshow

In [12]:
def LBPH(image, eps=1e-7):

		lbp = feature.local_binary_pattern(image, numPoints,
			radius, method="uniform")
		(hist, _) = np.histogram(lbp.ravel(),
			bins=np.arange(0, numPoints + 3),
			range=(0, numPoints + 2))

		hist = hist.astype("float")
		hist /= (hist.sum() + eps)

		return hist

In [13]:
numPoints = 24
radius = 8
detector = cv2.CascadeClassifier("/content/machine_learning/Classification/Face_Recognition/haarcascade_frontalface_default.xml")

data = []
labels = []

In [ ]:
for imagePath in paths.list_images("/content/machine_learning/Classification/Face_Recognition/data/Train"):
	image = cv2.imread(imagePath)
	print (imagePath)
	gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
	faceRects = detector.detectMultiScale(gray, scaleFactor=1.05, minNeighbors=5, minSize=(30, 30), flags=cv2.CASCADE_SCALE_IMAGE)
	if len(faceRects)>0:
		for (x, y, w, h) in faceRects:
			face_img = gray[y:y+h,x:x+w]
			hist = LBPH(face_img)
			labels.append(imagePath.split("/")[-2])
			data.append(hist)

In [15]:
model = RandomForestClassifier(n_estimators=20, random_state=42)
model.fit(data, labels)
f = open("classifier.cPickle", "wb")
f.write(cPickle.dumps(model))
f.close()

In [16]:
loaded_model = cPickle.load(open("classifier.cPickle", 'rb'))

In [ ]:
for imagePath in paths.list_images("/content/machine_learning/Classification/Face_Recognition/data/val"):
	image = cv2.imread(imagePath)
	gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
	faceRects = detector.detectMultiScale(gray, scaleFactor=1.05, minNeighbors=5, minSize=(30, 30), flags=cv2.CASCADE_SCALE_IMAGE)
	if len(faceRects)>0:
		for (x, y, w, h) in faceRects:
			face_img = gray[y:y+h,x:x+w]
			cv2.rectangle(image, (x, y), (x + w, y + h), (255,0, 0), 3)
			hist = LBPH(face_img)
			prediction = loaded_model.predict(hist.reshape(1, -1))[0]
			cv2.putText(image, prediction, (10, 30), cv2.FONT_HERSHEY_SIMPLEX,1.0, (0, 0, 255), 3)
			cv2_imshow(image)